
# Monet-Style CycleGAN Image Transformation

This notebook uses a pre-trained CycleGAN model to transform photos into Monet-style images.
Below, we document each step of the pipeline, provide optimized data handling, and introduce evaluation metrics for the generated images.



## 1. Data Loading and Preparation

The first step involves loading the dataset and preprocessing it for the model. We use TensorFlow's `tf.data` for efficient handling.


In [ ]:

import tensorflow as tf
import numpy as np

# Load and preprocess dataset
def preprocess_image(image):
    image = tf.image.resize(image, [256, 256])
    image = (image - 127.5) / 127.5  # Normalize to [-1, 1]
    return image

def load_dataset(dataset_path):
    # Example: Load dataset from a directory
    dataset = tf.keras.utils.image_dataset_from_directory(dataset_path, label_mode=None)
    dataset = dataset.map(preprocess_image).batch(32)
    return dataset

photo_ds = load_dataset('/path/to/photos')  # Update with actual path



## 2. Loading the Pre-Trained CycleGAN Model

We load the pre-trained CycleGAN generator to perform Monet-style transformations.


In [ ]:

from tensorflow.keras.models import load_model

# Load the pre-trained generator (replace with actual model path)
monet_generator = load_model('/path/to/pretrained/monet_generator.h5')



## 3. Visualizing Transformations

We display the original photos alongside the generated Monet-style images for comparison.


In [ ]:

import matplotlib.pyplot as plt

def visualize_transformations(photo_dataset, generator_model, num_samples=5):
    _, ax = plt.subplots(num_samples, 2, figsize=(10, 5 * num_samples))
    for i, img in enumerate(photo_dataset.take(num_samples)):
        prediction = generator_model(img, training=False)[0].numpy()
        prediction = (prediction * 127.5 + 127.5).astype(np.uint8)
        img = (img[0] * 127.5 + 127.5).numpy().astype(np.uint8)

        ax[i, 0].imshow(img)
        ax[i, 1].imshow(prediction)
        ax[i, 0].set_title("Input Photo")
        ax[i, 1].set_title("Monet-Style Output")
        ax[i, 0].axis("off")
        ax[i, 1].axis("off")
    plt.tight_layout()
    plt.show()

visualize_transformations(photo_ds, monet_generator)



## 4. Evaluating Generated Images

To quantitatively evaluate the transformations, we use Structural Similarity Index (SSIM) and Mean Squared Error (MSE).


In [ ]:

from skimage.metrics import structural_similarity as ssim
from sklearn.metrics import mean_squared_error

def evaluate_images(original_images, generated_images):
    ssim_scores = []
    mse_scores = []

    for original, generated in zip(original_images, generated_images):
        original = (original * 127.5 + 127.5).astype(np.uint8)
        generated = (generated * 127.5 + 127.5).astype(np.uint8)
        
        # Calculate SSIM
        score = ssim(original, generated, multichannel=True)
        ssim_scores.append(score)
        
        # Calculate MSE
        mse_score = mean_squared_error(original.flatten(), generated.flatten())
        mse_scores.append(mse_score)

    print(f"Average SSIM: {np.mean(ssim_scores):.4f}")
    print(f"Average MSE: {np.mean(mse_scores):.2f}")

# Example usage
original_images = [img.numpy() for img in photo_ds.take(5)]
generated_images = [monet_generator(img, training=False).numpy() for img in photo_ds.take(5)]
evaluate_images(original_images, generated_images)



## 5. Generating Monet-Style Images

We parallelize the image generation process to improve efficiency.


In [ ]:

import os
from concurrent.futures import ThreadPoolExecutor
from PIL import Image

output_dir = "../images"
os.makedirs(output_dir, exist_ok=True)

def save_image(img, output_path):
    img = (img * 127.5 + 127.5).astype(np.uint8)
    im = Image.fromarray(img)
    im.save(output_path)

def generate_images(photo_dataset, generator_model, output_dir):
    with ThreadPoolExecutor() as executor:
        for i, img in enumerate(photo_dataset):
            prediction = generator_model(img, training=False)[0].numpy()
            output_path = os.path.join(output_dir, f"{i + 1}.jpg")
            executor.submit(save_image, prediction, output_path)

generate_images(photo_ds, monet_generator, output_dir)



## 6. Archiving Submission Files

Finally, we create a zip archive of the generated images for submission.


In [ ]:

import shutil

shutil.make_archive("../images", 'zip', "../images")
print("Archive created at '../images.zip'")
